In [0]:
import requests
import json
import os


DATABRICKS_HOST = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

TARGET_AUTO_STOP_MINS = 1

HEADERS = {
    'Authorization': f'Bearer {DATABRICKS_TOKEN}',
    'Content-Type': 'application/json'
}

LIST_URL = f"{DATABRICKS_HOST}/api/2.0/sql/warehouses"


def list_warehouses():    
    try:
        response = requests.get(LIST_URL, headers=HEADERS)
        response.raise_for_status()
        return response.json().get('warehouses', [])
    except requests.exceptions.RequestException as e:
        print(f"❌ Error listing warehouses: {e}")
        return []

def update_warehouse_setting(warehouse):
    """Updates the auto_stop_mins setting for a single warehouse."""
    warehouse_id = warehouse.get('id')
    warehouse_name = warehouse.get('name')
    current_auto_stop = warehouse.get('auto_stop_mins')
    payload = {
        "id": warehouse_id,
        "name": warehouse_name,
        "cluster_size": warehouse.get('cluster_size', 'Medium'),
        "min_num_clusters": warehouse.get('min_num_clusters', 1),
        "max_num_clusters": warehouse.get('max_num_clusters', 1),
        "auto_stop_mins": TARGET_AUTO_STOP_MINS
        # Add other configurations (e.g., enable_photon) if they are consistently used
    }
    
    edit_url = f"{DATABRICKS_HOST}/api/2.0/sql/warehouses/{warehouse_id}/edit"
    
    try:
        response = requests.post(edit_url, headers=HEADERS, data=json.dumps(payload))
        response.raise_for_status()
        print(f"   Success: '{warehouse_name}' updated.")
    except requests.exceptions.HTTPError as err:
        print(f"   ❌ Error updating '{warehouse_name}': Selected Minimum Not Supported")
    except requests.exceptions.RequestException as e:
        print(f"   ❌ Network error updating '{warehouse_name}': {e}")


# --- Main Execution ---

all_warehouses = list_warehouses()

if not all_warehouses:
    print("No SQL Warehouses found or failed to retrieve the list. Script finished.")
else:
    print(f"Found {len(all_warehouses)} SQL Warehouses to process.")    
    for warehouse in all_warehouses:
        update_warehouse_setting(warehouse)